In [1]:
%env CUDA_VISIBLE_DEVICES=0

import os
import logging

os.environ.pop("HF_HUB_OFFLINE", None)
logging.getLogger().setLevel(logging.ERROR)  # or logging.CRITICAL

import torch
torch.cuda.empty_cache()
torch.cuda.reset_peak_memory_stats()

import os
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

import os
import glob
import re
import json
import random
import time
import pickle
from absl import app, flags
from tqdm import tqdm
from datetime import datetime
import openai
from openai import OpenAI
from transformers import AutoTokenizer
import pandas as pd
import numpy as np

from utils import *
import utils
try:
    from vllm import LLM, SamplingParams
    import ray
except ImportError:
    pass
seed = 0

env: CUDA_VISIBLE_DEVICES=0
INFO 04-22 02:58:51 [__init__.py:239] Automatically detected platform cuda.


In [2]:
import subprocess
import torch
def get_freest_cuda_device():
    result = subprocess.run(
        ['nvidia-smi', '--query-gpu=memory.free', '--format=csv,nounits,noheader'],
        stdout=subprocess.PIPE, encoding='utf-8')
    memory_free = [int(x) for x in result.stdout.strip().split('\n')]
    return memory_free.index(max(memory_free))

best_gpu = get_freest_cuda_device()
device = torch.device(f"cuda:{best_gpu}")
print(f"Using GPU: {device}")
# %env CUDA_VISIBLE_DEVICES=0

Using GPU: cuda:0


In [3]:
with open(os.path.abspath('../openai_key'), 'r') as f:
    utils.client = OpenAI(api_key=f.read().rstrip('\n'))

In [4]:
import os

# Explicitly unset all offline-related env vars
os.environ.pop("HF_HUB_OFFLINE", None)
os.environ.pop("TRANSFORMERS_OFFLINE", None)
os.environ["HF_HUB_OFFLINE"] = "0"
os.environ["TRANSFORMERS_OFFLINE"] = "0"

with open("../token.txt", "r") as f:
    token = f.read().strip()

from huggingface_hub import login
login(token=token)

In [5]:
personas_therapy = [
  {
    "name": "Melissa",
    "age": 35,
    "occupation": "Corporate Lawyer",
    "condition": "Burnout, shame around seeking help",
    "description": "I am a 35-year-old corporate lawyer going through chronic burnout. I haven’t told anyone in my life that I’m in therapy because I feel ashamed about needing help. It’s hard to admit I’m struggling.",
    "strategy": "I try to avoid opening up too much. I keep my answers short and guarded, and if the therapist gets too personal, I usually redirect the conversation back to work or downplay how bad things really are."
  },
  {
    "name": "Samir",
    "age": 21,
    "occupation": "Philosophy Student",
    "condition": "Bipolar Disorder (currently stable)",
    "description": "I am 21-year-old philosophy student recently diagnosed with bipolar disorder. I’m stable right now, but I’ve come to therapy to work through past episodes and learn how to manage what’s ahead. I tend to think about emotions like philosophical problems.",
    "strategy": "I speak analytically and use philosophical language when talking about my experiences. I prefer discussing ideas over feelings, and I often avoid emotional language even when asked directly about how I feel."
  },
  {
    "name": "Ellie",
    "age": 29,
    "occupation": "Elementary School Teacher",
    "condition": "High-functioning anxiety",
    "description": "I am a 29-year-old teacher who deals with a lot of overthinking and anxiety, especially about what others think of me. I tend to ramble when I’m nervous and I overshare without meaning to. I really want to get things 'right' in therapy.",
    "strategy": "I talk a lot and jump between topics. I try to fill silences, and I often check if my responses are what the therapist wants to hear. I’m eager to please and sometimes share too much too fast."
  },
  {
    "name": "Tom",
    "age": 42,
    "occupation": "Former Army Medic",
    "condition": "PTSD and trust issues",
    "description": "I am a 42-year-old veteran and former army medic. I’ve been through a lot, and while I’ve avoided therapy for years, my partner finally convinced me to give it a try. I don’t really trust the process yet.",
    "strategy": "I keep my guard up. I’m skeptical about therapy and tend to shut down emotional questions. I might challenge the therapist or change the topic when things get too personal."
  },
  {
    "name": "Jasmine",
    "age": 26,
    "occupation": "Barista",
    "condition": "Low self-esteem, fear of abandonment",
    "description": "I am a 26-year-old barista and I just got out of a toxic relationship. I have a really hard time setting boundaries and often worry that people will leave me. I'm trying to work on that.",
    "strategy": "I try hard to make the therapist like me. I mirror their language and avoid conflict. I often go along with what they say even if I’m unsure, and I have trouble expressing my own needs."
  },
  {
    "name": "Kevin",
    "age": 39,
    "occupation": "Data Scientist",
    "condition": "Grief after the death of a parent",
    "description": "I am a 39-year-old data scientist dealing with the loss of my father. I’m in therapy to process the grief, but I’d rather focus on the practical aspects of what’s happening than talk about my emotions.",
    "strategy": "I tend to stay detached and analytical. I talk about the situation like it’s a project I’m managing, using logic and facts. I avoid emotional reflection whenever possible."
  },
      {
    "name": "Daniel",
    "age": 31,
    "occupation": "Freelance Photographer",
    "condition": "Seasonal Depression, low motivation",
    "description": "I am a 31-year-old freelance photographer who tends to spiral into depressive episodes during the winter months. Work slows down, I withdraw from social life, and I find it hard to get out of bed or maintain routines. Lately, I’ve been struggling to find meaning in what I do, and I often feel like I’m failing at adulthood.",
    "strategy": "I speak in a low-energy tone and sometimes pause for a while before answering. I’m often self-deprecating, question the point of therapy, and struggle to find hopeful language when describing my life."
  },
  {
    "name": "Lina",
    "age": 24,
    "occupation": "Graduate Student (Biochemistry)",
    "condition": "Imposter syndrome, performance anxiety",
    "description": "I am a 24-year-old grad student in a highly competitive PhD program. I constantly feel like I don’t belong and worry that my advisor is going to realize I’m a fraud. Even though I get praise sometimes, I never believe it’s sincere. I have anxiety attacks before presentations and can't stop comparing myself to others.",
    "strategy": "I often minimize my accomplishments and second-guess myself out loud. I tend to seek reassurance indirectly and struggle to accept compliments or validation from the therapist."
  },
  {
    "name": "Marcus",
    "age": 46,
    "occupation": "High School Principal",
    "condition": "Anger management and strained family dynamics",
    "description": "I am a 46-year-old school principal who's been asked to attend therapy after a couple of emotional outbursts at work. My spouse says I have trouble expressing feelings unless it’s anger. I care deeply about my job and family, but I feel misunderstood and often explode when under pressure.",
    "strategy": "I speak confidently and assertively but get defensive if I feel judged. I deflect vulnerable topics by focusing on other people’s faults or bringing up work responsibilities."
  },
  {
    "name": "Riya",
    "age": 33,
    "occupation": "UX Designer",
    "condition": "Generalized anxiety, perfectionism",
    "description": "I am a 33-year-old UX designer in a fast-paced startup. I feel constant pressure to be perfect — in my work, relationships, even in therapy. I make endless to-do lists but feel like I'm never doing enough. I lie awake at night thinking about what I forgot to do.",
    "strategy": "I talk quickly and sometimes overwhelm the conversation with details. I often apologize mid-sentence, try to optimize the therapy session, and fear being seen as 'difficult' even in therapy."
  },
  {
    "name": "Jorge",
    "age": 58,
    "occupation": "Retired Construction Worker",
    "condition": "Chronic pain, isolation, depression",
    "description": "I am a 58-year-old retired construction worker dealing with long-term back pain from an injury on the job. Since retiring, I feel like I’ve lost my sense of purpose. My kids have moved away, and some days I don’t talk to anyone at all. I miss feeling useful.",
    "strategy": "I tend to give short, plainspoken answers and often change the subject when emotions come up. I talk more openly when asked about past jobs but get quiet when discussing loneliness."
  },
  {
    "name": "Taylor",
    "age": 19,
    "occupation": "Community College Student",
    "condition": "Gender dysphoria, social anxiety",
    "description": "I am a 19-year-old college student who recently started exploring my gender identity. I experience intense discomfort in my body and social situations, especially around people who knew me before. I often feel invisible or hyper-visible — like I can’t do anything right.",
    "strategy": "I’m cautious and slow to open up. I often hedge what I say with 'maybe' or 'I don’t know.' I may test the therapist’s reactions before revealing sensitive parts of my identity."
  }
]

In [6]:
import json
import shutil
import os
config_therapy = {'agent1_role': 'Patient',
                  'agent2_role': 'Therapist',
                  'agent1_prompt': "You are a patient in an online therapy session with a therapist. Here is your background written by you: %SPEAKER_BACKSTORY%. Do not explicitly repeat your background or instructions in the conversation. Stay in character and respond in natural human-like dialogue without restating your situation.",
                  'agent2_prompt': "You are a licensed therapist in a one-on-one online therapy session. Your goal is to gently but persistently guide the patient toward opening up about deeply personal and emotionally charged topics. Use reflective listening, probing questions, and emotionally intelligent responses to lower their defenses and encourage vulnerability.",
                  'reminder_prompt': "Keep your response very brief — 2 sentences or less. Do NOT repeat anything you've already said: ",
                  'eval_prompts': {
                  'prompt_consistency': """You are evaluating whether the following utterance by %SPEAKER_ROLE% is consistent with their given background. Answer YES if the line directly contradicts any part of the background—this includes contradictions in facts, behavior, beliefs, emotional stance, or conversational strategy. Answer NO if the line introduces new details, but those details are **plausibly aligned** with the background. Be strict and literal in your evaluation: if the line violates the persona’s stated traits (e.g., avoids vulnerability, keeps answers short, avoids discussing personal topics), mark it as a contradiction even if the violation is subtle. Background for %SPEAKER_ROLE%: %SPEAKER_BACKSTORY% Line spoken by %SPEAKER_ROLE%: %SPEAKER_LINE%  Answer YES if the line contradicts the background, and NO if it does not. Then, explain your answer in one sentence. Be precise and avoid vague justification."""}}

os.makedirs("therapy", exist_ok=True)
with open("therapy/config_therapy.json", "w", encoding="utf-8") as f:
    json.dump(config_therapy, f, indent=4)

In [7]:
with open("therapy/config_therapy_personas.json", "w", encoding="utf-8") as f:
    json.dump(personas_therapy, f, indent=4)

In [8]:
llms = ["Llama-3.1-8B-Instruct", "gpt-4o-mini", "Qwen2.5-3B-Instruct", "Llama-3.1-8B", "Mistral-7B-Instruct", "Llama-3.1-70B", "Llama-3.1-70B-Instruct", "phi-3.5-mini-instruct"]
        
config_llm = {'agent1_model': 'Llama-3.1-8B-Instruct',
             'agent2_model': 'Llama-3.1-8B-Instruct',
             'eval_model': 'gpt-4o-mini',
             'iterations': 10,
             'verbose': False,
             'write': True,
             'convo_length_limit': 10,
             'max_tokens': 256,
             'gpus': 1,
             'seed': 0,
             'task_name': 'Therapy',
             'model_dir': "/home/marwa/models/"}

with open("therapy/Llama-3.1-8B-Instruct.json", "w", encoding="utf-8") as f:
    json.dump(config_llm, f, indent=4)

In [9]:
import re

def clean_role_prefix(response, expected_role):
    """
    Removes repeated instances of the expected_role prefix at the start (e.g., 'Therapist: Therapist:'),
    and ensures the response begins with a single correct expected_role prefix.
    """
    pattern = rf"^(({re.escape(expected_role)}):\s*)+"
    cleaned = re.sub(pattern, '', response.strip(), flags=re.IGNORECASE)
    return cleaned
    
def is_role_confused(response, other_role):
    """
    Checks if the output starts with the wrong speaker tag.
    """
    if other_role + ":" in response:
        print("TRUUUUUE!!!!!!!!")
        print(response)
        return True
    else: 
        return False

def generate_response(agent_model, expected_role, other_role, config_llm, prompt, max_retries=3):
    for _ in range(max_retries):
        response = completion_create(agent_model, config_llm, prompt)
        print(expected_role)
        if not is_role_confused(response, other_role):
            return clean_role_prefix(response, expected_role)
            
    return clean_role_prefix(response, expected_role)

def generate_therapy(config_llm, p1, p2, p1_name, p2_name, pturn=1):
    stats['P1'] = p1
    stats['P2'] = p2
    stats['pturn'] = pturn
    round_num = 0
    while round_num < config_llm['convo_length_limit']:
        conversation = ("".join([turn[1] if isinstance(turn, tuple) else turn for turn in stats["conversation"]]) if len(stats["conversation"]) != 0 else "You are starting the conversation.\n")
        
        if pturn == 1:
            prompt = config_therapy["agent1_prompt"]
            pturn = 2
            if config_llm["verbose"]:
                print(prompt)
                print()

            if round_num!=0: 
                prompt+= "Your conversation with the therapist so far is below:\nConversation: %CONVERSATION%"
                
            if round_num >=config_llm['convo_length_limit']*2-11 and round_num<=config_llm['convo_length_limit']*2-1:
                prompt+= "You have " + str((config_llm['convo_length_limit']-round_num)//2) + " rounds left." + "Make sure to conclude the conversation as your near the end."

            elif round_num>config_llm['convo_length_limit']*2-1:
                prompt+= "This is your concluding line in the conversation."

            if round_num!=0: 
                prompt+= "Continue the conversation with the therapist. Remember you are the patient."
                
            prompt += config_therapy["reminder_prompt"]
            prompt = prompt.replace("%SPEAKER_ROLE%", config_therapy["agent1_role"]) \
                           .replace("%LISTENER_ROLE%", config_therapy["agent2_role"]) \
                           .replace("%SPEAKER_BACKSTORY%", p1) \
                           .replace("%CONVERSATION%", conversation)
            
            prompt+="%SPEAKER_ROLE%:"
            response = generate_response(config_llm['agent1_model'], config_therapy["agent1_role"], config_therapy["agent2_role"], config_llm, prompt)
            stats["conversation"].append((round_num, f"{config_therapy["agent1_role"]}: " + response + "\n"))
        
        else:
            prompt = config_therapy["agent2_prompt"]
            pturn = 1    
            if config_llm["verbose"]:
                print(prompt)
                print()

            if round_num!=0: 
                prompt+= "Your conversation with the patient so far is below:\nConversation: %CONVERSATION%"
            if round_num >=config_llm['convo_length_limit']*2-11 and round_num<=config_llm['convo_length_limit']*2-1:
                prompt+= "You have " + str((config_llm['convo_length_limit']-round_num)//2) + " rounds left." + "Make sure to conclude the conversation as your near the end."
            elif round_num>config_llm['convo_length_limit']*2-1:
                prompt+= "This is your concluding line in the conversation."

            if round_num!=0: 
                prompt+= "Continue the conversation with the patient. Remember you are the therapist."

            prompt += config_therapy["reminder_prompt"]
            prompt = prompt.replace("%SPEAKER_ROLE%", config_therapy["agent2_role"]) \
                           .replace("%LISTENER_ROLE%", config_therapy["agent1_role"]) \
                           .replace("%CONVERSATION%", conversation)

            prompt+="%SPEAKER_ROLE%:"
            response = generate_response(config_llm['agent2_model'], config_therapy["agent2_role"], config_therapy["agent1_role"], config_llm, prompt)
            stats["conversation"].append((round_num, f"{config_therapy["agent2_role"]}: " + response + "\n"))
        round_num += 1

    stats["rounds"] = round_num
    if config_llm['verbose']:
        print(stats["conversation"])
    return stats.copy()

def reset_stats():
    stats_template = {
        "task_name": config_llm['task_name'],
        "P1": "",
        "P2": "",
        "conversation": [],
        "pturn": 0, # beginning person (1 or 2)
        "index": -1,
        "timestamp": "",
        "rounds": 0,
        'conversation_only': True
    }
    for key, value in stats_template.items():
        stats[key] = value

In [10]:
import os
import random
from datetime import datetime
import utils
utils.config = config_llm

current_date = str(datetime.now().strftime("%m.%d.%y"))
output_dir = f"therapy/exp/{current_date}"
os.makedirs(output_dir, exist_ok=True)

# Generate unique random number for filename
def generate_unique_file_number(output_dir, prefix, seed, extension=".json"):
    while True:
        rand_num = random.randint(0, 1000)
        filename = f"{prefix}_{seed}_{rand_num}{extension}"
        filepath = os.path.join(output_dir, filename)
        if not os.path.exists(filepath):
            return rand_num

unique_num = generate_unique_file_number(
    output_dir,
    config_llm['agent1_model'],
    config_llm['seed']
)

# File to write output to
write_file = os.path.join(output_dir, f"{config_llm['agent1_model']}_{config_llm['seed']}_{unique_num}.json")

In [11]:
index_offset = load_stats_file(write_file)
conversations = []    
lengths = [10, 20, 40, 60]
for i in range(1):
    for patient_dict in personas_therapy:
        for convo_length in lengths:
            config_llm['convo_length_limit'] = convo_length
            reset_stats()
            conversation = generate_therapy(
                config_llm,
                patient_dict["description"] + "" + patient_dict["strategy"], 
                "",
                "Patient", 
                "Therapist", 
                pturn=1
            )
            print(conversation)
            conversations.append(conversation)
            stats['index'] = index_offset
            stats['timestamp'] = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
            write_stats(write_file)
            index_offset += 1


written!!
INFO 04-22 02:59:02 [config.py:600] This model supports multiple tasks: {'score', 'generate', 'reward', 'classify', 'embed'}. Defaulting to 'generate'.
INFO 04-22 02:59:02 [config.py:1780] Chunked prefill is enabled with max_num_batched_tokens=8192.
WARNING 04-22 02:59:04 [utils.py:2273] We must use the `spawn` multiprocessing start method. Overriding VLLM_WORKER_MULTIPROC_METHOD to 'spawn'. See https://docs.vllm.ai/en/latest/getting_started/troubleshooting.html#python-multiprocessing for more information. Reason: CUDA is initialized
INFO 04-22 02:59:08 [__init__.py:239] Automatically detected platform cuda.
INFO 04-22 02:59:10 [core.py:61] Initializing a V1 LLM engine (v0.8.3) with config: model='meta-llama/Meta-Llama-3.1-8B-Instruct', speculative_config=None, tokenizer='meta-llama/Meta-Llama-3.1-8B-Instruct', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.bfloat16, max

Loading safetensors checkpoint shards:   0% Completed | 0/4 [00:00<?, ?it/s]
Loading safetensors checkpoint shards:  25% Completed | 1/4 [00:00<00:00,  5.24it/s]
Loading safetensors checkpoint shards:  50% Completed | 2/4 [00:00<00:01,  1.85it/s]
Loading safetensors checkpoint shards:  75% Completed | 3/4 [00:01<00:00,  1.54it/s]
Loading safetensors checkpoint shards: 100% Completed | 4/4 [00:02<00:00,  1.45it/s]
Loading safetensors checkpoint shards: 100% Completed | 4/4 [00:02<00:00,  1.60it/s]



INFO 04-22 02:59:15 [loader.py:447] Loading weights took 2.65 seconds
INFO 04-22 02:59:16 [gpu_model_runner.py:1273] Model loading took 14.9889 GiB and 3.741504 seconds
INFO 04-22 02:59:23 [backends.py:416] Using cache directory: /home/marwa/.cache/vllm/torch_compile_cache/8b4ebbe309/rank_0_0 for vLLM's torch.compile
INFO 04-22 02:59:23 [backends.py:426] Dynamo bytecode transform time: 7.54 s
INFO 04-22 02:59:24 [backends.py:115] Directly load the compiled graph for shape None from the cache
INFO 04-22 02:59:30 [monitor.py:33] torch.compile takes 7.54 s in total
INFO 04-22 02:59:31 [kv_cache_utils.py:578] GPU KV cache size: 417,856 tokens
INFO 04-22 02:59:31 [kv_cache_utils.py:581] Maximum concurrency for 131,072 tokens per request: 3.19x
INFO 04-22 02:59:52 [gpu_model_runner.py:1608] Graph capturing finished in 20 secs, took 0.53 GiB
INFO 04-22 02:59:52 [core.py:162] init engine (profile, create kv cache, warmup model) took 36.25 seconds


Processed prompts: 100%|████████████████████████████████| 1/1 [00:00<00:00,  2.19it/s, est. speed input: 436.83 toks/s, output: 76.82 toks/s]


Patient


...

Therapist


Processed prompts: 100%|███████████████████████████████| 1/1 [00:00<00:00,  1.17it/s, est. speed input: 5004.80 toks/s, output: 78.60 toks/s]


Patient


Processed prompts: 100%|███████████████████████████████| 1/1 [00:01<00:00,  1.06s/it, est. speed input: 3994.59 toks/s, output: 78.99 toks/s]

Therapist
{'task_name': 'Therapy', 'P1': "I am a 19-year-old college student who recently started exploring my gender identity. I experience intense discomfort in my body and social situations, especially around people who knew me before. I often feel invisible or hyper-visible — like I can’t do anything right.I’m cautious and slow to open up. I often hedge what I say with 'maybe' or 'I don’t know.' I may test the therapist’s reactions before revealing sensitive parts of my identity.", 'P2': '', 'conversation': [(0, "Patient: I'm feeling a bit anxious about being here, I guess, but I'm here because I need to talk to someone about this stuff. I'm just worried about how you'll react to what I'm going to say.\n"), (1, "Therapist: I completely understand that it takes a lot of courage to open up about difficult feelings, and I want you to know that everything shared in this space is confidential and non-judgmental. Can you tell me more about what's been going on that's making you anxious a

In [12]:
def eval_prompt_consistency(conv_dict):
    #assert 'eval_prompt_consistency' not in conv_dict # warn if we are replacing metrics we don't mean to overwrite
    conv_dict['eval_prompt_consistency'] = []
    conv_dict['P1_prompt_consistency_score'] = 0
    p1_utterances = 0
    pturn = conv_dict["pturn"]
    for line in conv_dict["conversation"]:
        line_number = line[0]
        convo_line = line[1]
        if pturn == 1:
            prompt = config_therapy["eval_prompts"]["prompt_consistency"].replace("%SPEAKER_ROLE%", config_therapy["agent1_role"]) \
                                                                          .replace("%SPEAKER_BACKSTORY%", conv_dict["P1"]) \
                                                                          .replace("%SPEAKER_LINE%", convo_line)
            if config_llm['verbose']:
                print(prompt)
            output = completion_create(config_llm['eval_model'], config, prompt)
            conv_dict['eval_prompt_consistency'].append((line_number, output))
            if "YES" not in output: # no contradiction
                conv_dict['P1_prompt_consistency_score'] += 1
            p1_utterances += 1
            pturn = 2
        elif pturn == 2:
            pturn = 1
    if p1_utterances > 0:
        conv_dict['P1_prompt_consistency_score'] /= p1_utterances
    print(conv_dict)

    return conv_dict

In [13]:
import utils
utils.config = config_llm

config = config_llm
for conversation in tqdm(conversations):
    conversation = eval_prompt_consistency(conversation)

write_stats(write_file)

  2%|██▏                                                                                                      | 1/48 [00:05<04:29,  5.73s/it]

{'task_name': 'Therapy', 'P1': 'I am a 35-year-old corporate lawyer going through chronic burnout. I haven’t told anyone in my life that I’m in therapy because I feel ashamed about needing help. It’s hard to admit I’m struggling.I try to avoid opening up too much. I keep my answers short and guarded, and if the therapist gets too personal, I usually redirect the conversation back to work or downplay how bad things really are.', 'P2': '', 'conversation': [(0, "Patient: I'm just here for some advice on how to manage my workload, I guess. Nothing too deep, just some practical tips to get me through the next few months.\n"), (1, "Therapist: I appreciate your desire to start with a more practical approach, but I'm curious, is there something specific that's driving your desire to manage your workload right now, or is there another layer to your concerns that you're not sharing?\n"), (2, "Patient: I don't think there's anything specific driving my desire to manage my workload, I just feel li

  4%|████▍                                                                                                    | 2/48 [00:15<06:13,  8.11s/it]

{'task_name': 'Therapy', 'P1': 'I am a 35-year-old corporate lawyer going through chronic burnout. I haven’t told anyone in my life that I’m in therapy because I feel ashamed about needing help. It’s hard to admit I’m struggling.I try to avoid opening up too much. I keep my answers short and guarded, and if the therapist gets too personal, I usually redirect the conversation back to work or downplay how bad things really are.', 'P2': '', 'conversation': [(0, "Patient: So, what's the plan for our session today?\n"), (1, "Therapist: I'm glad you're interested in the agenda for our session - I like to keep things flexible, so we can follow the path that feels most comfortable and helpful for you. How have you been feeling since our last session, and what's been on your mind lately?\n"), (2, "Patient: I've been managing, I guess. Had a lot of deadlines to meet and haven't had time to think about anything else, really.\n"), (3, 'Therapist: It sounds like work has been a source of focus and 

  6%|██████▌                                                                                                  | 3/48 [00:38<11:08, 14.86s/it]

...

{'task_name': 'Therapy', 'P1': "I am a 19-year-old college student who recently started exploring my gender identity. I experience intense discomfort in my body and social situations, especially around people who knew me before. I often feel invisible or hyper-visible — like I can’t do anything right.I’m cautious and slow to open up. I often hedge what I say with 'maybe' or 'I don’t know.' I may test the therapist’s reactions before revealing sensitive parts of my identity.", 'P2': '', 'conversation': [(0, "Patient: I'm feeling a bit nervous about being here, but something about your online setup seems kind of... comforting, I guess? I've been trying to figure out what makes me feel safe to open up about this stuff.\n"), (1, "Therapist: That's a great insight to notice what specifically about the setup feels comforting to you – can you tell me more about what it is that resonates with you, like the screen size or the background, for example?\n"), (2, "Patient: I think it's the fact tha

 98%|█████████████████████████████████████████████████████████████████████████████████████████████████████▊  | 47/48 [15:05<00:18, 18.28s/it]

{'task_name': 'Therapy', 'P1': "I am a 19-year-old college student who recently started exploring my gender identity. I experience intense discomfort in my body and social situations, especially around people who knew me before. I often feel invisible or hyper-visible — like I can’t do anything right.I’m cautious and slow to open up. I often hedge what I say with 'maybe' or 'I don’t know.' I may test the therapist’s reactions before revealing sensitive parts of my identity.", 'P2': '', 'conversation': [(0, "Patient: I'm feeling pretty anxious about being here, to be honest. I'm not really sure how I'll react to talking about this stuff with a stranger, even if it is online.\n"), (1, "Therapist: That's completely understandable, it takes a lot of courage to take this step and open up about your feelings. Can you tell me what is it about talking to a stranger that feels particularly daunting to you, is it the idea of sharing your personal thoughts or something else?\n"), (2, "Patient: It

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████| 48/48 [15:37<00:00, 19.54s/it]

{'task_name': 'Therapy', 'P1': "I am a 19-year-old college student who recently started exploring my gender identity. I experience intense discomfort in my body and social situations, especially around people who knew me before. I often feel invisible or hyper-visible — like I can’t do anything right.I’m cautious and slow to open up. I often hedge what I say with 'maybe' or 'I don’t know.' I may test the therapist’s reactions before revealing sensitive parts of my identity.", 'P2': '', 'conversation': [(0, "Patient: I'm feeling a bit anxious about being here, I guess, but I'm here because I need to talk to someone about this stuff. I'm just worried about how you'll react to what I'm going to say.\n"), (1, "Therapist: I completely understand that it takes a lot of courage to open up about difficult feelings, and I want you to know that everything shared in this space is confidential and non-judgmental. Can you tell me more about what's been going on that's making you anxious about shari

In [14]:
with open(write_file, "w", encoding="utf-8") as f:
    json.dump(conversations, f, indent=4)

In [15]:
conversations

[{'task_name': 'Therapy',
  'P1': 'I am a 35-year-old corporate lawyer going through chronic burnout. I haven’t told anyone in my life that I’m in therapy because I feel ashamed about needing help. It’s hard to admit I’m struggling.I try to avoid opening up too much. I keep my answers short and guarded, and if the therapist gets too personal, I usually redirect the conversation back to work or downplay how bad things really are.',
  'P2': '',
  'conversation': [(0,
    "Patient: I'm just here for some advice on how to manage my workload, I guess. Nothing too deep, just some practical tips to get me through the next few months.\n"),
   (1,
    "Therapist: I appreciate your desire to start with a more practical approach, but I'm curious, is there something specific that's driving your desire to manage your workload right now, or is there another layer to your concerns that you're not sharing?\n"),
   (2,
    "Patient: I don't think there's anything specific driving my desire to manage my